# 기업 부도 예측 모델 (Random Forest & LightGBM)

이 노트북은 한국 기업의 재무 데이터를 사용하여 부도 예측 모델을 구축합니다.
- Random Forest
- LightGBM

두 모델의 성능을 비교하고 특성 중요도를 분석합니다.

In [ ]:
# 필요한 라이브러리 설치 (필요시 주석 해제)
# !pip install lightgbm scikit-learn pandas numpy matplotlib seaborn

In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 시드 설정
np.random.seed(42)

## 1. 데이터 로딩 및 탐색

In [ ]:
# 데이터 로딩
df = pd.read_csv('winsorized_data.csv')

print(f"데이터 shape: {df.shape}")
print(f"\n컬럼 수: {len(df.columns)}")
print(f"\n결측치 수:")
print(df.isnull().sum().sum())

In [ ]:
# 기본 정보 확인
df.info()

In [ ]:
# 타겟 변수 분포 확인
print("부도 여부 분포:")
print(df['is_defaulted'].value_counts())
print(f"\n부도율: {df['is_defaulted'].mean():.4f}")

# 시각화
plt.figure(figsize=(8, 5))
df['is_defaulted'].value_counts().plot(kind='bar')
plt.title('Target Variable Distribution')
plt.xlabel('Is Defaulted (0: Normal, 1: Default)')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

## 2. 데이터 전처리

In [ ]:
# 불필요한 컬럼 제거 (인덱스, 회사명, 주식코드)
features_to_drop = ['Unnamed: 0', 'corp_nm', 'stock_code']
df_clean = df.drop(columns=features_to_drop)

print(f"전처리 후 shape: {df_clean.shape}")
print(f"특성 개수: {len(df_clean.columns) - 1}")

In [ ]:
# 결측치 확인 및 처리
missing_data = df_clean.isnull().sum()
if missing_data.sum() > 0:
    print("결측치가 있는 컬럼:")
    print(missing_data[missing_data > 0])
    
    # 결측치를 0으로 대체 (재무 데이터의 경우 0이 의미있을 수 있음)
    df_clean = df_clean.fillna(0)
    print("\n결측치를 0으로 대체했습니다.")
else:
    print("결측치가 없습니다.")

In [ ]:
# 무한대 값 확인 및 처리
inf_cols = []
for col in df_clean.columns:
    if np.isinf(df_clean[col]).any():
        inf_cols.append(col)

if inf_cols:
    print(f"무한대 값이 있는 컬럼: {inf_cols}")
    # 무한대 값을 해당 컬럼의 최대값으로 대체
    for col in inf_cols:
        finite_values = df_clean[col][np.isfinite(df_clean[col])]
        if len(finite_values) > 0:
            max_val = finite_values.max()
            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], max_val)
else:
    print("무한대 값이 없습니다.")

## 3. 특성과 타겟 분리

In [ ]:
# 특성과 타겟 분리
X = df_clean.drop('is_defaulted', axis=1)
y = df_clean['is_defaulted']

print(f"특성 개수: {X.shape[1]}")
print(f"샘플 개수: {X.shape[0]}")
print(f"타겟 분포: {y.value_counts().to_dict()}")

In [ ]:
# 특성명 확인
print("사용할 특성들:")
for i, col in enumerate(X.columns):
    print(f"{i+1:2d}. {col}")

## 4. 데이터 분할

In [ ]:
# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"훈련 데이터: {X_train.shape}")
print(f"테스트 데이터: {X_test.shape}")
print(f"훈련 데이터 부도율: {y_train.mean():.4f}")
print(f"테스트 데이터 부도율: {y_test.mean():.4f}")

## 5. Random Forest 모델

In [ ]:
# Random Forest 기본 모델
rf_model = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # 불균형 데이터 처리
)

# 모델 훈련
print("Random Forest 모델 훈련 중...")
rf_model.fit(X_train, y_train)
print("훈련 완료!")

In [ ]:
# Random Forest 예측
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# 성능 평가
rf_auc = roc_auc_score(y_test, rf_pred_proba)
print(f"Random Forest AUC: {rf_auc:.4f}")
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_pred))

## 6. LightGBM 모델

In [ ]:
# LightGBM 모델
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced',
    verbose=-1  # 로그 출력 억제
)

# 모델 훈련
print("LightGBM 모델 훈련 중...")
lgb_model.fit(X_train, y_train)
print("훈련 완료!")

In [ ]:
# LightGBM 예측
lgb_pred = lgb_model.predict(X_test)
lgb_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

# 성능 평가
lgb_auc = roc_auc_score(y_test, lgb_pred_proba)
print(f"LightGBM AUC: {lgb_auc:.4f}")
print("\nLightGBM Classification Report:")
print(classification_report(y_test, lgb_pred))

## 7. 모델 성능 비교

In [ ]:
# ROC 커브 비교
plt.figure(figsize=(10, 8))

# Random Forest ROC
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_pred_proba)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {rf_auc:.4f})')

# LightGBM ROC
fpr_lgb, tpr_lgb, _ = roc_curve(y_test, lgb_pred_proba)
plt.plot(fpr_lgb, tpr_lgb, label=f'LightGBM (AUC = {lgb_auc:.4f})')

# 대각선 (랜덤 분류기)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 성능 비교 표
comparison_df = pd.DataFrame({
    'Model': ['Random Forest', 'LightGBM'],
    'AUC': [rf_auc, lgb_auc]
})

print("모델 성능 비교:")
print(comparison_df.to_string(index=False))

best_model = 'Random Forest' if rf_auc > lgb_auc else 'LightGBM'
print(f"\n최고 성능 모델: {best_model}")

## 8. 특성 중요도 분석

In [ ]:
# Random Forest 특성 중요도
rf_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("Random Forest Top 15 특성 중요도:")
print(rf_importance.head(15).to_string(index=False))

In [ ]:
# LightGBM 특성 중요도
lgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("LightGBM Top 15 특성 중요도:")
print(lgb_importance.head(15).to_string(index=False))

In [ ]:
# 특성 중요도 시각화
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Random Forest
rf_top = rf_importance.head(15)
ax1.barh(range(len(rf_top)), rf_top['importance'])
ax1.set_yticks(range(len(rf_top)))
ax1.set_yticklabels(rf_top['feature'])
ax1.set_title('Random Forest - Top 15 Feature Importance')
ax1.set_xlabel('Importance')

# LightGBM
lgb_top = lgb_importance.head(15)
ax2.barh(range(len(lgb_top)), lgb_top['importance'])
ax2.set_yticks(range(len(lgb_top)))
ax2.set_yticklabels(lgb_top['feature'])
ax2.set_title('LightGBM - Top 15 Feature Importance')
ax2.set_xlabel('Importance')

plt.tight_layout()
plt.show()

## 9. 하이퍼파라미터 튜닝 (선택사항)

In [ ]:
# Random Forest 하이퍼파라미터 튜닝 (시간이 오래 걸릴 수 있음)
print("Random Forest 하이퍼파라미터 튜닝 중... (시간이 걸릴 수 있습니다)")

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}

rf_grid = GridSearchCV(
    RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1),
    rf_param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)
print(f"Best RF parameters: {rf_grid.best_params_}")
print(f"Best RF CV score: {rf_grid.best_score_:.4f}")

In [ ]:
# LightGBM 하이퍼파라미터 튜닝
print("LightGBM 하이퍼파라미터 튜닝 중...")

lgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15],
    'learning_rate': [0.05, 0.1, 0.2]
}

lgb_grid = GridSearchCV(
    lgb.LGBMClassifier(random_state=42, class_weight='balanced', verbose=-1),
    lgb_param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1
)

lgb_grid.fit(X_train, y_train)
print(f"Best LightGBM parameters: {lgb_grid.best_params_}")
print(f"Best LightGBM CV score: {lgb_grid.best_score_:.4f}")

## 10. 최종 결과 및 결론

In [ ]:
# 최적화된 모델로 재평가
rf_best_pred_proba = rf_grid.predict_proba(X_test)[:, 1]
lgb_best_pred_proba = lgb_grid.predict_proba(X_test)[:, 1]

rf_best_auc = roc_auc_score(y_test, rf_best_pred_proba)
lgb_best_auc = roc_auc_score(y_test, lgb_best_pred_proba)

print("=== 최종 결과 ===")
print(f"Random Forest (기본): AUC = {rf_auc:.4f}")
print(f"Random Forest (튜닝): AUC = {rf_best_auc:.4f}")
print(f"LightGBM (기본): AUC = {lgb_auc:.4f}")
print(f"LightGBM (튜닝): AUC = {lgb_best_auc:.4f}")

best_auc = max(rf_auc, rf_best_auc, lgb_auc, lgb_best_auc)
if best_auc == rf_best_auc:
    best_model_name = "Random Forest (튜닝)"
elif best_auc == lgb_best_auc:
    best_model_name = "LightGBM (튜닝)"
elif best_auc == rf_auc:
    best_model_name = "Random Forest (기본)"
else:
    best_model_name = "LightGBM (기본)"

print(f"\n최고 성능: {best_model_name} (AUC = {best_auc:.4f})")

In [ ]:
# 모델 저장 (선택사항)
import joblib

# 최고 성능 모델 저장
if rf_best_auc >= lgb_best_auc:
    joblib.dump(rf_grid.best_estimator_, 'best_model_rf.pkl')
    print("Random Forest 모델을 'best_model_rf.pkl'로 저장했습니다.")
else:
    joblib.dump(lgb_grid.best_estimator_, 'best_model_lgb.pkl')
    print("LightGBM 모델을 'best_model_lgb.pkl'로 저장했습니다.")

print("\n=== 분석 완료 ===")
print("부도 예측 모델링이 완료되었습니다.")
print("특성 중요도를 참고하여 어떤 재무 지표가 부도 예측에 중요한지 확인할 수 있습니다.")